# Haensel AMS - Predicting Price

## Contents
* [Assignment](#Assignment)
* [Question 1](#Question-1)
* [Question 2](#Question-2)
* [Question 3](#Question-3)
* [Question 4](#Question-4)

### Assignment 
The target variable is 'price'. You have 7 attributes and obviously we want you to build some initial ML model which predicts the prices.

1. Make some initial data analysis, which will hint to some stuctures in the data and how attributes are connected.
2. Fit some ML model(s) and explain briefly your choices.
3. Show with some X-validation the power of your model and comment the results.
4. Present us the results and the steps you have taken and also with some critical thinking and next steps.


## Question 1

1. Make some initial data analysis, which will hint to some stuctures in the data and how attributes are connected.

In [1]:
import pandas as pd
import numpy as np

In [43]:
data = pd.read_csv('sample.csv')
data.head(3)

,loc1,loc2,para1,dow,para2,para3,para4,price
0,0,01,1,Mon,662,3000.0,3.8,73.49
1,9,99,1,Thu,340,2760.0,9.2,300.00
2,0,04,0,Mon,16,2700.0,3.0,130.00


In [3]:
data.isnull().sum()

loc1     0
loc2     0
para1    0
dow      0
para2    0
para3    0
para4    0
price    0
dtype: int64

We don't have any missing value(NaN). 

In [48]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9993 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   loc1    9993 non-null   float64
 1   loc2    9993 non-null   float64
 2   para1   9993 non-null   int64  
 3   dow     9993 non-null   object 
 4   para2   9993 non-null   int64  
 5   para3   9993 non-null   float64
 6   para4   9993 non-null   float64
 7   price   9993 non-null   float64
dtypes: float64(5), int64(2), object(1)
memory usage: 702.6+ KB


In [5]:
data.describe()

,para1,para2,para3,para4,price
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,1.380800,447.384000,9547.989920,8.459522,433.733056
std,3.500831,221.005861,8022.814037,4.613526,277.435947
min,0.000000,16.000000,200.000000,1.000000,50.730000
25%,1.000000,301.000000,2898.000000,4.000000,250.000000
50%,1.000000,434.000000,6447.000000,7.200000,370.000000
75%,1.000000,582.000000,15000.000000,13.600000,550.000000
max,337.000000,2554.000000,34782.000000,27.200000,5700.000000


In [35]:
data['loc1'].value_counts()

2    1607
0    1486
1    1223
7    1081
3     945
5     846
4     773
8     727
9     690
6     620
S       1
T       1
Name: loc1, dtype: int64

In [36]:
data = data[(data['loc1'].str.contains('S') == False) & (data['loc1'].str.contains('T') == False)]

In [37]:
data['loc1'].value_counts()

2    1607
0    1486
1    1223
7    1081
3     945
5     846
4     773
8     727
9     690
6     620
Name: loc1, dtype: int64

We add errors argument equal to coerce to change string types to Na. Then we will use the dropna method to remove Na's.

In [47]:
data['loc1'] = pd.to_numeric(data['loc1'], errors='coerce')
data['loc2'] = pd.to_numeric(data['loc2'], errors='coerce')
data.dropna(inplace = True)

In [51]:
dow_dummies = pd.get_dummies(data['dow'])
dow_dummies.head()

,Fri,Mon,Sat,Sun,Thu,Tue,Wed
0,0,1,0,0,0,0,0
1,0,0,0,0,1,0,0
2,0,1,0,0,0,0,0
3,0,1,0,0,0,0,0
4,0,0,0,0,1,0,0


In [53]:
data_2 = data.copy(deep=True)

In [54]:
data_2.drop('dow', axis = 1,inplace = True)
result = data_2.join(dow_dummies)
result.head()

,loc1,loc2,para1,para2,para3,para4,price,Fri,Mon,Sat,Sun,Thu,Tue,Wed
0,0.0,1.0,1,662,3000.0,3.8,73.49,0,1,0,0,0,0,0
1,9.0,99.0,1,340,2760.0,9.2,300.00,0,0,0,0,1,0,0
2,0.0,4.0,0,16,2700.0,3.0,130.00,0,1,0,0,0,0,0
3,4.0,40.0,1,17,12320.0,6.4,365.00,0,1,0,0,0,0,0
4,5.0,50.0,1,610,2117.0,10.8,357.50,0,0,0,0,1,0,0


In [55]:
days_of_week = {'Mon': 1, 'Tue' : 2, 'Wed' : 3, 'Thu' : 4, 'Fri' : 5, 'Sat' : 6, 'Sun' : 7} 

In [56]:
data['dow'] = data['dow'].map(days_of_week)
data.head()

,loc1,loc2,para1,dow,para2,para3,para4,price
0,0.0,1.0,1,1,662,3000.0,3.8,73.49
1,9.0,99.0,1,4,340,2760.0,9.2,300.00
2,0.0,4.0,0,1,16,2700.0,3.0,130.00
3,4.0,40.0,1,1,17,12320.0,6.4,365.00
4,5.0,50.0,1,4,610,2117.0,10.8,357.50
